In [ ]:
# # Volume aggregation for Fyers API data
# import os
# import glob

# fyers_storage_path = r'D:\Programming\Download_Backtest_Deploy_data\1__Download\1__Download_data_Fyers_via_API\nifty50_daily_data'

# # Start with NIFTY 50 Date and Volume columns as base
# fyers_volume_df = nifty_50_df[['Date', 'Volume']].copy()
# fyers_volume_df['Date'] = pd.to_datetime(fyers_volume_df['Date'])

# # Aggregate volume data for each stock from Fyers API
# for file in glob.glob(os.path.join(fyers_storage_path, '*_daily_data.csv')):
#     symbol = os.path.basename(file).replace('_daily_data.csv', '').replace('-', '_')
#     symbol_df = pd.read_csv(file)
#     symbol_df['datetime'] = pd.to_datetime(symbol_df['datetime'])
#     symbol_df['Date'] = symbol_df['datetime'].dt.date
#     symbol_df['Date'] = pd.to_datetime(symbol_df['Date'])
    
#     # Merge volume data
#     merged = fyers_volume_df.merge(symbol_df[['Date', 'volume']], on='Date', how='left')
#     fyers_volume_df[f'{symbol}_v'] = merged['volume']

# fyers_volume_df

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
nifty_50_df = pd.read_csv(r'D:\Programming\Download_Backtest_Deploy_data\1__Download\TradingView_data_download\NIFTY_50_historical_data.csv')[:-1]


In [ ]:
import os
import glob

storage_path = r'D:\Programming\Download_Backtest_Deploy_data\1__Download\TradingView_data_download\storage'

nifty_50_components_df = nifty_50_df[['Date', 'Volume']].copy()

for file in glob.glob(os.path.join(storage_path, '*_historical_data.csv')):
    symbol = os.path.basename(file).replace('_historical_data.csv', '')
    symbol_df = pd.read_csv(file)[:-1]
    symbol_df['Date'] = pd.to_datetime(symbol_df['Date'])
    nifty_50_components_df['Date'] = pd.to_datetime(nifty_50_components_df['Date'])
    
    merged = nifty_50_components_df.merge(symbol_df[['Date', 'Volume']], on='Date', how='left', suffixes=('', f'_{symbol}'))
    nifty_50_components_df[f'{symbol}_v'] = merged[f'Volume_{symbol}']

df = nifty_50_components_df

print(df.shape)
df.head()

In [ ]:
# Load and display the data in proper JSON format
json_file_path = r'D:\Programming\Download_Backtest_Deploy_data\1__Download\TradingView_data_download\nifty50_data.json'

with open(json_file_path, 'r') as f:
    loaded_data = json.load(f)

nifty_50_components_list = loaded_data['nifty50_components']


In [ ]:
for action in loaded_data['nifty50_components_changes']:
    print(action)

In [ ]:
nifty_50_components_list = ['Date', 'Volume'] + nifty_50_components_list

df_v = df[nifty_50_components_list].copy()
df_v['total_volume'] = np.nan

volume_cols = [col for col in df_v.columns if col not in ['Date', 'Volume', 'total_volume']]


n = 0


for i in range(len(df_v) -1, -1, -1):

    current_date = str(df_v['Date'].iloc[i].date().strftime('%Y-%m-%d'))

    # Check for corporate_actions on this current_date
    for ca_action in loaded_data['corporate_actions']:
        if ca_action['effective_date'] == current_date:


            n += 1 
            if n > 2:
                break


            symbol = ca_action['symbol'] + "_v"

            factor = 1.0

            if ca_action['action_type'] == "bonus":
                a, b = ca_action['ratio'].split(":")
                a, b = int(a), int(b)
                factor = 1/((a+b)/b)

            if ca_action['action_type'] == "stock_split":
                x, y = ca_action['ratio'].split(":")
                x, y = int(x), int(y)
                factor = 1/(x/y)

            # special case
            if ca_action['action_type'] == "demerger":
                # print("I am here in demerger")
                factor = 0.9439368770000008

            i_ch = -1 if ca_action['action_type'] != "demerger" else 0
            mask_date = df_v['Date'].iloc[df_v[df_v['Date'] == current_date + " 09:15:00"].index[0] + i_ch]

            mask = (df_v['Date'] >= '2021-01-01') & (df_v['Date'] < mask_date)
            df_v.loc[mask, symbol] = df_v.loc[mask, symbol] * factor
            
            print(ca_action, factor)

    # Check for nifty50_components_changes on this current_date
    for action in loaded_data['nifty50_components_changes']:
        if action['effective_date'] == current_date:
            
            for stock_remove in action['added']:
                print(stock_remove)
                volume_cols.remove(stock_remove + '_v')
                
            for stock_add in action['removed']:
                print(stock_add)
                volume_cols.append(stock_add + '_v')
                volume_cols = sorted(set(volume_cols))
                df_v[stock_add + '_v'] = df[stock_add + '_v']

    # ============= special volume changes =============

    special_volume = 0.0

    if current_date in ['2025-04-08', '2025-04-09', '2025-04-11']:
        special_volume -= df_v['ETERNAL_v'].iloc[i]

    single_row_total = df_v[volume_cols].iloc[i].sum() + special_volume

    df_v.loc[i, 'total_volume'] = single_row_total

# mask = (df_v['Date'] >= '2020-03-27') & (df_v['Date'] < '2021-03-31')
# df_v.loc[mask, 'HDFCBANK_v'] = df_v.loc[mask, 'HDFCBANK_v'] / 2

# ===========================================

n = 0

for i in range(len(df_v) -1, -1, -1):
    if df_v['total_volume'].iloc[i] == df_v['Volume'].iloc[i]:
        resu = 'Matched' 
        n += 1
    else:
        resu = 'Not Matched' 
      
    print(f"{i:4d} | {df_v['Date'].iloc[i].date().strftime('%Y-%m-%d')} | {resu:11s} | Total: {df_v['total_volume'].iloc[i]:>13,.0f} | NIFTY: {df_v['Volume'].iloc[i]:>13,.0f}")

print(f"\nSummary: {n} out of {len(df_v)} rows matched ({n/len(df_v)*100:.1f}%)")


In [85]:
x = 0
for i in range(1103, 1100, -1):

    resu = 'Matched' if df_v['total_volume'].iloc[i] == df_v['Volume'].iloc[i]  else 'Not Matched' 

    analysis = ""

    if resu == 'Not Matched':

        diff = df_v['Volume'].iloc[i] - df_v['total_volume'].iloc[i] #- (df_v['WIPRO_v'].iloc[i] * 0.5)
        analysis = f"{diff}"

        x = i

    print(f"{i:4d} | {df_v['Date'].iloc[i].date().strftime('%Y-%m-%d')} | {resu:11s} | Original NIFTY: {df_v['Volume'].iloc[i]:>13} | Total: {df_v['total_volume'].iloc[i]:>13} | {analysis}")

az = df_v[df_v['Date'] == df_v['Date'].iloc[x]]
az

1103 | 2025-06-16 | Matched     | Original NIFTY:   305813047.0 | Total:   305813047.0 | 
1102 | 2025-06-13 | Matched     | Original NIFTY:   327875182.0 | Total:   327875182.0 | 
1101 | 2025-06-12 | Not Matched | Original NIFTY:   328310748.0 | Total:   339460245.0 | -11149497.0


,Date,Volume,ADANIENT_v,ADANIPORTS_v,APOLLOHOSP_v,ASIANPAINT_v,AXISBANK_v,BAJAJFINSV_v,BAJAJ_AUTO_v,BAJFINANCE_v,BEL_v,BHARTIARTL_v,CIPLA_v,COALINDIA_v,DRREDDY_v,EICHERMOT_v,ETERNAL_v,GRASIM_v,HCLTECH_v,HDFCBANK_v,HDFCLIFE_v,HEROMOTOCO_v,HINDALCO_v,HINDUNILVR_v,ICICIBANK_v,INDUSINDBK_v,INFY_v,ITC_v,JIOFIN_v,JSWSTEEL_v,KOTAKBANK_v,LT_v,M&M_v,MARUTI_v,NESTLEIND_v,NTPC_v,ONGC_v,POWERGRID_v,RELIANCE_v,SBILIFE_v,SBIN_v,SHRIRAMFIN_v,SUNPHARMA_v,TATACONSUM_v,TATAMOTORS_v,TATASTEEL_v,TCS_v,TECHM_v,TITAN_v,TRENT_v,ULTRACEMCO_v,WIPRO_v,total_volume,BPCL_v,BRITANNIA_v,DIVISLAB_v,LTIM_v,UPL_v,SHREECEM_v,IOC_v,GAIL_v
1101,2025-06-12 09:15:00,328310748.00,955097.00,2192452.00,552007.00,3417375.00,3461744.00,3783347.00,368162.00,12388330.00,15555771.00,7346031.00,1199034.00,11262656.00,2625058.00,337217.00,24298795.00,545570.00,3607962.00,16349130.00,1600355.00,813687.00,6539340.00,3339584.00,7953247.00,2742462.00,10483574.00,18508835.00,16175003.00,2223026.00,2978172.00,2335718.00,2278398.00,347339.00,1358579.00,11881618.00,35613271.00,14071159.00,8828114.00,984693.00,13448521.00,4970089.00,3588365.00,941294.00,12585149.00,27846151.00,4060134.00,2316173.00,811822.00,974013.00,262530.00,14528657.00,339460245.00,13568848.00,305696.00,647651.00,339679.00,1097714.00,36013.00,30425949.00,27331751.00


In [88]:
6373575.0 / 7081750.00

0.9

In [ ]:
abs(az['BAJFINANCE_v'].item() * 0.9), abs(diff)

In [ ]:
abs(az['BAJFINANCE_v'].item() * 0.9) == abs(diff)

In [ ]:
loaded_data

In [ ]:
loaded_data['nifty50_components_changes']

In [118]:
def get_nearby_dates(all_effective_dates_list, target_date_str, days_range=6):

    target_date = datetime.strptime(target_date_str, "%Y-%m-%d").date()
    effective_dates = [datetime.strptime(d, "%Y-%m-%d").date() for d in all_effective_dates_list]
    
    nearby_dates = [
        d for d in effective_dates
        if abs((d - target_date).days) <= days_range
    ]
    
    nearby_dates.sort(key=lambda d: abs((d - target_date).days))
    
    return list(set([d.strftime("%Y-%m-%d") for d in nearby_dates]))

# ----------------

all_effective_dates_list = []

for field in loaded_data:
    if field == 'nifty50_components': continue
    for sub_field in loaded_data[field]:

        all_effective_dates_list.append(sub_field['effective_date'])

result = get_nearby_dates(all_effective_dates_list, "2022-09-15")

# ----------------

found_actions_list = [] 
for field in loaded_data:
    if field == 'nifty50_components': continue
    for sub_field in loaded_data[field]:
        for res_i in result:
            if sub_field['effective_date'] == res_i:
                found_actions_list.append(sub_field)

fac_symbol = set()
for x in found_actions_list: 
    print(x)
    fac_symbol.add(x['symbol'] + "_v")

fac_symbol = list(fac_symbol)

factor_list = []
for fac_sy in fac_symbol: factor_list.append(abs(diff) / az[fac_sy].item())
factor_list

{'symbol': 'BAJAJFINSV', 'action_type': 'stock_split', 'ratio': '5:1', 'effective_date': '2022-09-13'}
{'symbol': 'BAJAJFINSV', 'action_type': 'bonus', 'ratio': '1:1', 'effective_date': '2022-09-13'}
{'symbol': 'BEL', 'action_type': 'bonus', 'ratio': '2:1', 'effective_date': '2022-09-15'}


[0.7167434516746228, 2.9469929668095474]